# Import Neighboorhoods file

This Docker container may serve as a starting point for your course project.
It includes a set of simple instructions to:
<br>
 -> load a dataset (locally)
 <br>
 -> into a database
 <br>
 -> and be able to query data from the database
 <br>
 -> and make simple visualizations on the queried data

In [1]:
# Imports

from sqlalchemy import create_engine, text, inspect, Table
import pandas as pd

## Load csv file

Load the file called world_population.csv into a pandas dataframe. Make sure you parse the columns correctly.

In [2]:
import os

# Get the current working directory
current_directory = os.getcwd()

# Get the parent directory (directory above the current directory)
parent_directory = os.path.dirname(current_directory)

# List all folders in the parent directory
folders_in_parent_directory = [folder for folder in os.listdir(parent_directory) if os.path.isdir(os.path.join(parent_directory, folder))]

# Print the list of folders
print("Folders in the parent directory:")
for folder in folders_in_parent_directory:
    print(folder)


Folders in the parent directory:
etc
proc
root
mnt
home
boot
dev
opt
lib
srv
sys
usr
media
var
bin
sbin
tmp
run
notebook
data


In [11]:
# Load the csv into a pandas dataframe (https://www.w3schools.com/python/pandas/pandas_dataframes.asp)
pets_df = pd.read_csv("../data/Pets.csv")

print(pets_df)

owners_df = pd.read_csv("../data/Owners.csv", delimiter=",")
print (owners_df.head())

procedures_df = pd.read_csv("../data/ProceduresHistory.csv", delimiter=",")
print (procedures_df.head())

      petid     name    kind  gender  age  ownerid
0   J6-8562  Blackie     Dog    male   11     5168
1   Q0-2001   Roomba     Cat    male    9     5508
2   M0-2904    Simba     Cat    male    1     3086
3   R3-7551   Keller  Parrot  female    2     7908
4   P2-7342  Cuddles     Dog    male   13     4378
..      ...      ...     ...     ...  ...      ...
95  U8-6473  Biscuit     Dog  female    3     1070
96  I5-4893   Cookie     Cat  female    3     7340
97  Q8-0954  Lakshmi     Cat  female    7     9385
98  N0-9539  Swiffer     Cat    male   14     9365
99  S5-5938      Taz     Dog    male    6     9427

[100 rows x 6 columns]
   ownerid     name     surname          streetaddress                 city  \
0     6049   Debbie    Metivier        315 Goff Avenue         Grand Rapids   
1     2863     John   Sebastian      3221 Perry Street              Davison   
2     3518   Connie      Pauley  1539 Cunningham Court  Bloomfield Township   
3     3663     Lena  Haliburton   4217 Twin Oaks

## Store data into database
Save the contents in the world_population file to the a table called population in the database. 

In [21]:
engine = create_engine("postgresql://student:infomdss@db_dashboard:5432/dashboard")


with engine.connect() as conn:
    conn.execute(text("DROP TABLE IF EXISTS pets CASCADE;"))
    conn.execute(text("DROP TABLE IF EXISTS owners CASCADE;"))
    conn.execute(text("DROP TABLE IF EXISTS procedures CASCADE;"))

pets_df.to_sql("pets", engine, if_exists="replace", index=True)
owners_df.to_sql("owners", engine, if_exists="replace", index=True)
procedures_df.to_sql("procedures", engine, if_exists="replace", index=True)

284

## Fetch data from database
Read the table **population** from the database in a dataframe. Make sure the index column is the index of the dataframe.

In [27]:
procedures_table = pd.read_sql_table('procedures', engine, index_col='index')
print(procedures_table)

procedures_table = pd.read_sql_table('pets', engine, index_col='index')
print(procedures_table)

procedures_table = pd.read_sql_table('owners', engine, index_col='index')
print(procedures_table)

         petid        date    proceduretype  proceduresubcode
index                                                        
0      A8-1181  2016-01-10     VACCINATIONS                 5
1      E7-3766  2016-01-11     VACCINATIONS                 5
2      B8-8740  2016-01-11     VACCINATIONS                 5
3      D4-9443  2016-01-11     VACCINATIONS                 5
4      F6-3398  2016-01-12  HOSPITALIZATION                 1
...        ...         ...              ...               ...
2279   A4-2573  2016-12-30     VACCINATIONS                 5
2280   B3-8317  2016-12-30     VACCINATIONS                 5
2281   C5-4291  2016-12-30       ORTHOPEDIC                 3
2282   C6-3299  2016-12-31     VACCINATIONS                 5
2283   A1-3879  2016-12-31     VACCINATIONS                 5

[2284 rows x 4 columns]
         petid     name    kind  gender  age  ownerid
index                                                
0      J6-8562  Blackie     Dog    male   11     5168
1      

In [33]:
procedures = pd.read_sql('SELECT * FROM pets INNER JOIN procedures ON pets.petid = procedures.petid INNER JOIN owners ON pets.ownerid = owner.ownerid', engine)
print(procedures)

ProgrammingError: (psycopg2.errors.UndefinedTable) missing FROM-clause entry for table "owner"
LINE 1: ...cedures.petid INNER JOIN owners ON pets.ownerid = owner.owne...
                                                             ^

[SQL: SELECT * FROM pets INNER JOIN procedures ON pets.petid = procedures.petid INNER JOIN owners ON pets.ownerid = owner.ownerid]
(Background on this error at: https://sqlalche.me/e/20/f405)